# Milutin Popovic

In [1]:
import numpy as np
import numdifftools as nd

In [2]:
def lagrange_newton(f, h_s, x_0, mu_0, eps, kmax):
    mu_l = [mu_0]; x_l = [x_0]
    n = len(x_0); p = len(mu_0)
    x_k = x_0; mu_k = mu_0
    x_mu_k = np.concatenate([x_k, mu_k])
    
    f_hess = nd.Hessian(f)
    f_grad = nd.Gradient(f)
    h_hess_s = [nd.Hessian(h_i) for h_i in h_s]
    h_grad_s = [nd.Gradient(h_i) for h_i in h_s]
    
    for k in range(kmax):
        # Construct
        d_L_k = f_grad(x_k) + sum(mu_k[i]*h_grad_s[i](x_k) for i in range(p))
        dd_L_k = f_hess(x_k) + sum(mu_k[i]*h_hess_s[i](x_k) for i in range(p))
        
        if p == 1:
            h_k = np.array([h_s[0](x_k)])
            nabl_h_k = h_grad_s[0](x_k)
            part_2 = np.hstack([nabl_h_k, np.zeros(p)])
        else:
            h_k = np.hstack([h_s[i](x_k) for i in range(p)])
            nabl_h_k = np.vstack([h_grad_s[i](x_k) for i in range(p)])
            part_2 = np.hstack([nabl_h_k, np.zeros([p, p])])
        
        part_1 = np.hstack([dd_L_k , nabl_h_k.reshape(n, p)])
        nabl_phi =  np.vstack([part_1, part_2])
        
        phi = np.concatenate([d_L_k, h_k])
        
        if np.linalg.norm(phi) <= eps:
            return x_l, mu_l, k
        
        d_x_mu_k = np.linalg.solve(nabl_phi, -phi)
        x_mu_k = x_mu_k + d_x_mu_k
        
        
        x_l.append(x_mu_k[:n])
        mu_l.append(x_mu_k[n:])
        k += 1
    
    return x_l, mu_l, kmax

In [3]:
f_a = lambda x: 2*x[0]**4 + x[1]**4 + 4*x[0]**2 - x[0]*x[1] + 6*x[1]**2
h_a = lambda x: 2*x[0] - x[1] + 4
x_0 = np.array([0, 0]); mu_0 = np.array([0]); kmax = 200; eps = 10e-3

x_l, mu_l, k = lagrange_newton(f_a, [h_a], x_0, mu_0, eps, kmax)
print('x_k: ', x_l[-1])
print('μ_k: ', mu_l[-1])
print('k :', k)

x_k:  [-353.84615385   92.30769231]
μ_k:  [1461.53846154]
k : 200


In [4]:
f_b = lambda x: 1000 - x[0]**2 - 2*x[1]**2 - x[2]**2 - x[0]*x[1] - x[0]*x[2]
h_1 = lambda x: x[0]**2 + x[1]**2 + x[2]**2 - 25
h_2 = lambda x: 8*x[0] + 14*x[1] - 7*x[2] - 56
x_0 = np.array([3, 0.2, 3]); mu_0 = np.array([0, 0]); kmax = 200; eps = 10e-5

x_l, mu_l, k = lagrange_newton(f_b, [h_1, h_2], x_0, mu_0, eps, kmax)
print('x_k: ', x_l[-1])
print('μ_k: ', mu_l[-1])
print('k :', k)

x_k:  [-754.14996676 1591.53061069  886.06125938]
μ_k:  [435.38303733 469.48428151]
k : 200
